## Criação de dados da Escala

In [1]:
from datetime import date
from source.classes.escala import Escala
from source.classes.calendario import Calendario
from source.classes.tipoAtividade import TipoAtividade, TipoAtividadeComplementar, TipoAtividadeVazia
from source.classes.atividade import Atividade
from source.enums.tipoMemoria import TipoMemoria
from source.enums.turno import Turno


In [2]:
atividade_ppp = TipoAtividade("Pré-parto, parto e pós-parto", "PPP", 6, TipoMemoria.PRATICA)
atividade_cch = TipoAtividadeComplementar("PPP")
atividade_mpi = TipoAtividade("Metodologia de pesquisa I", "MPI", 6, TipoMemoria.TEORICA)

In [3]:
atividade_1 = Atividade(atividade_ppp, Turno.MANHA, date(2025, 6, 1))
atividade_2 = Atividade(atividade_cch, Turno.TARDE, date(2025, 6, 2))
atividade_3 = Atividade(atividade_mpi, Turno.TARDE, date(2025, 6, 1))
atividades = [atividade_1, atividade_2, atividade_3]
atividades.append(Atividade(atividade_ppp, Turno.MANHA, date(2025, 6, 3)))
atividades.append(Atividade(atividade_ppp, Turno.MANHA, date(2025, 6, 4)))
atividades.append(Atividade(atividade_ppp, Turno.MANHA, date(2025, 6, 5)))
atividades.append(Atividade(atividade_ppp, Turno.MANHA, date(2025, 6, 6)))
atividades.append(Atividade(atividade_mpi, Turno.TARDE, date(2025, 6, 7)))

In [4]:
calendario = Calendario(date(2025, 6, 1), atividades)
print(calendario)

Calendário início 2025-06-01, fim 2025-06-30:
- Dia: 1 D, PPP - Pré-parto, parto e pós-parto, tipo P  duração 6 horas, turno M
- Dia: 1 D, MPI - Metodologia de pesquisa I, tipo T  duração 6 horas, turno T
- Dia: 1 D, Atividade Vazia (sem carga horária), turno N
- Dia: 2 S, Atividade Vazia (sem carga horária), turno M
- Dia: 2 S, CCH - Complementação de carga horária (PPP), tipo P  duração 12 horas, turno T
- Dia: 2 S, Atividade Vazia (sem carga horária), turno N
- Dia: 3 T, PPP - Pré-parto, parto e pós-parto, tipo P  duração 6 horas, turno M
- Dia: 3 T, Atividade Vazia (sem carga horária), turno T
- Dia: 3 T, Atividade Vazia (sem carga horária), turno N
- Dia: 4 Q, PPP - Pré-parto, parto e pós-parto, tipo P  duração 6 horas, turno M
- Dia: 4 Q, Atividade Vazia (sem carga horária), turno T
- Dia: 4 Q, Atividade Vazia (sem carga horária), turno N
- Dia: 5 Q, PPP - Pré-parto, parto e pós-parto, tipo P  duração 6 horas, turno M
- Dia: 5 Q, Atividade Vazia (sem carga horária), turno T
- Dia

In [5]:
atividade_30 = Atividade(atividade_ppp, Turno.NOITE, date(2025, 6, 30))
atividade_31 = Atividade(atividade_cch, Turno.NOITE, date(2025, 6, 30))
calendario.adicionar_atividade(atividade_30)
calendario.adicionar_atividade(atividade_31)
# print(calendario)

Substituindo atividade vazia no dia 2025-06-30 turno Turno.NOITE
Atividade anterior: Dia: 30 S, Atividade Vazia (sem carga horária), turno N
Nova atividade: Dia: 30 S, PPP - Pré-parto, parto e pós-parto, tipo P  duração 6 horas, turno N


In [6]:
escala = Escala(
    instituicao="Maternidade Bárbara Heliodora",
    residente="Milene Mendes da Silva",
    setores=["PPP"],
    preceptores=["Enfª Obst. Ana Cláudia"],
    calendario=calendario
)

print(escala)

Escala de Milene Mendes da Silva na Maternidade Bárbara Heliodora:
Setores: PPP
Preceptores: Enfª Obst. Ana Cláudia
Calendário: Calendário início 2025-06-01, fim 2025-06-30:
- Dia: 1 D, PPP - Pré-parto, parto e pós-parto, tipo P  duração 6 horas, turno M
- Dia: 1 D, MPI - Metodologia de pesquisa I, tipo T  duração 6 horas, turno T
- Dia: 1 D, Atividade Vazia (sem carga horária), turno N
- Dia: 2 S, Atividade Vazia (sem carga horária), turno M
- Dia: 2 S, CCH - Complementação de carga horária (PPP), tipo P  duração 12 horas, turno T
- Dia: 2 S, Atividade Vazia (sem carga horária), turno N
- Dia: 3 T, PPP - Pré-parto, parto e pós-parto, tipo P  duração 6 horas, turno M
- Dia: 3 T, Atividade Vazia (sem carga horária), turno T
- Dia: 3 T, Atividade Vazia (sem carga horária), turno N
- Dia: 4 Q, PPP - Pré-parto, parto e pós-parto, tipo P  duração 6 horas, turno M
- Dia: 4 Q, Atividade Vazia (sem carga horária), turno T
- Dia: 4 Q, Atividade Vazia (sem carga horária), turno N
- Dia: 5 Q, PPP

### Leitura da imagem

In [121]:
import io
import base64
from PIL import Image
import pytesseract
import numpy as np
import cv2

VALORES_VALIDOS = {'T', 'D', 'PN', 'PD'}


def processa_foto_forca_bruta(
    imagem=None,
    imagem_base64=None,
    n_colunas=31,
    default_linha1='T',
    default_linha2='D'
):
    """
    Processa a imagem de tabela, corta em duas linhas, divide em colunas iguais e valida célula por célula.

    Args:
        imagem: arquivo (Django)
        imagem_base64: dataURL da imagem
        n_colunas: número de colunas (28–31)
        default_linha1: valor default para 1ª linha em caso de texto inválido
        default_linha2: valor default para 2ª linha em caso de texto inválido

    Returns:
        (linha1, linha2): listas com os valores extraídos/validados
    """
    if imagem_base64:
        if "," in imagem_base64:
            imagem_base64 = imagem_base64.split(",")[1]
        image_bytes = io.BytesIO(base64.b64decode(imagem_base64))
    elif imagem:
        image_bytes = io.BytesIO()
        for chunk in imagem.chunks():
            image_bytes.write(chunk)
        image_bytes.seek(0)
    else:
        return "Nenhuma imagem fornecida."

    # Carrega imagem
    file_bytes = np.asarray(bytearray(image_bytes.read()), dtype=np.uint8)
    img_cv = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

    if img_cv is None:
        return "Erro: Não foi possível decodificar a imagem."

    altura, largura = img_cv.shape[:2]
    meio_y = altura // 2

    cell_width = largura / n_colunas
    cell_height = meio_y

    linha1 = []
    linha2 = []

    # para cada coluna
    for i in range(n_colunas):
        x_start = int(i * cell_width)
        x_end = int((i + 1) * cell_width)

        # --- primeira linha ---
        cell1 = img_cv[0:meio_y, x_start:x_end]
        val1 = ocr_celula(cell1)
        if val1 in VALORES_VALIDOS:
            linha1.append(val1)
        elif val1 != '':
            linha1.append(default_linha1)
        else:
            linha1.append('')

        # --- segunda linha ---
        cell2 = img_cv[meio_y:altura, x_start:x_end]
        val2 = ocr_celula(cell2)
        if val2 in VALORES_VALIDOS:
            linha2.append(val2)
        elif val2 != '':
            linha2.append(default_linha2)
        else:
            linha2.append('')

    return linha1, linha2

def ocr_celula(celula_cv, min_conf=60):
    """
    Executa OCR em uma célula e retorna texto limpo se confiança for aceitável.
    """

    img_pil = Image.fromarray(celula_cv)

    # OCR detalhado
    data = pytesseract.image_to_data(
        img_pil,
        lang='por',
        config='--psm 10',  # ideal para 1 caractere
        output_type=pytesseract.Output.DICT
    )

    n = len(data['text'])
    for i in range(n):
        texto = data['text'][i].strip().upper()
        try:
            conf = float(data['conf'][i])
        except ValueError:
            conf = -1

        if texto != '' and conf >= min_conf:
            return texto

    return ''



In [122]:
linha1, linha2 = processa_foto_forca_bruta(
    imagem_base64=img_base64,
    n_colunas=30,
    default_linha1='T',
    default_linha2='PN'
)

print(linha1)
print(linha2)



['', 'T', '', 'T', '', '', 'T', '', 'T', '', '', '', '', '', '', 'T', 'T', '', '', 'T', '', '', '', 'T', 'T', '', '', '', '', '']
['PN', '', 'PN', 'PN', '', '', 'PN', 'PN', '', '', 'PN', 'PN', 'PN', '', '', 'PN', 'PN', '', 'PN', '', 'PN', 'PN', '', 'PN', '', '', 'PN', 'PN', '', 'PN']
